In [1]:
# import all the necessary libraries
import openai, pyperclip, os
from rich import print
import threading
import tkinter as tk
from tkinter import ttk

In [3]:
# you can get your API_KEY from this link: https://platform.openai.com/account/api-keys

API_KEY = 'paste your api key'

In [ ]:
class OpenAI:
    def __init__(self, api_key) -> None:
        openai.api_key = api_key
        self.response = ''
        self.prompt = ''
        
    def generate_response(self, prompt: str) -> str:
        self.prompt = prompt
        completion = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=[{"role": "user", "content": prompt}])
        self.response = completion.choices[0].message.content
        return self.response

    def handle_exception(self, error: str) -> str:
        if isinstance(error, openai.error.OpenAIError):
            print(f'[bold red] Response had a error: {error}[/bold red]')
        elif isinstance(error, IndexError):
            print(f'[bold red] Response had a error: no response generated[/bold red]')
        elif isinstance(error, Exception):
            print(f'[bold red]Error: {error}[/bold red]')
        else:
            print(error)
            

In [ ]:
def search(api, entry, text, prompt_label):
    prompt = entry.get()

    # Create the progress bar widget in the same frame as the search bar
    progress = ttk.Progressbar(top_frame, mode='indeterminate', length=170)
    progress.place(relx=0.5, rely=1.4, anchor=tk.CENTER)
    progress.start()

    def generate_response_thread():
        try:
            response = api.generate_response(prompt)
            root.after(0, update_response, response)
        except Exception as error:
            root.after(0, api.handle_exception, error)

    def update_response(response):
        text.delete(1.0, tk.END)
        text.insert(tk.END, response)
        pyperclip.copy(response)
        entry.delete(0, tk.END)  # Clear the search bar after a successful search
        prompt_label.configure(text=f"Prompt: {api.prompt}")
        progress.stop()
        progress.place_forget()

    threading.Thread(target=generate_response_thread).start()


In [5]:

if __name__ == "__main__":
    api = OpenAI(API_KEY)

    root = tk.Tk()
    root.title("Personal GPT-3.5 Turbo Application")
    root.resizable(False, False)

    top_frame = ttk.Frame(root, padding=10)
    top_frame.grid(row=0, column=0, sticky=tk.W)
    middle_frame = ttk.Frame(root, padding=10)
    middle_frame.grid(row=1, column=0, sticky=tk.W)
    bottom_frame = ttk.Frame(root, padding=10)
    bottom_frame.grid(row=2, column=0, sticky=tk.W)

    entry_label = ttk.Label(top_frame, text="Search:")
    entry = ttk.Entry(top_frame, width=40)
    search_button = ttk.Button(top_frame, text="Search", command=lambda: search(api, entry, response_text, prompt_label))
    prompt_label = ttk.Label(middle_frame, text="")
    response_label = ttk.Label(bottom_frame, text="Response:")
    response_text = tk.Text(bottom_frame, wrap=tk.WORD, width=50, height=15,bg='light blue')
    
    
    entry_label.grid(row=0, column=0, sticky=tk.W)
    entry.grid(row=0, column=1, sticky=tk.W)
    search_button.grid(row=0, column=2, sticky=tk.W)
    prompt_label.grid(row=0, column=0, sticky=tk.W)
    response_label.grid(row=0, column=0, sticky=tk.W)
    response_text.grid(row=1, column=0, columnspan=3, sticky=tk.W)
    

    root.mainloop()
